In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

def create_ssd_model(num_classes):
    base_model = tf.keras.applications.MobileNetV2(input_shape=(640, 640, 3), include_top=False)
    base_out = base_model.output
    x = layers.Conv2D(4 * (num_classes + 4), kernel_size=(3, 3), padding='same')(base_out)
    model = models.Model(inputs=base_model.input, outputs=x)
    return model

model = create_ssd_model(num_classes=26)  # Change num_classes according to your dataset


2024-04-01 14:50:04.301540: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-01 14:50:04.320800: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-01 14:50:04.422077: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-01 14:50:04.422128: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-01 14:50:04.441639: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

9406464/9406464 [==============================] - 1s 0us/step


In [11]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError

# Define your model architecture
def create_ssd_model(input_shape, num_classes):
    base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape, include_top=False)
    base_out = base_model.output
    x = layers.Conv2D(4 * (num_classes + 4), kernel_size=(3, 3), padding='same')(base_out)
    model = models.Model(inputs=base_model.input, outputs=x)
    return model

# Define your data loading pipeline
import json

def parse_coco_annotations(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)

    annotations = {}
    for image_data in data['images']:
        image_id = image_data['id']
        file_name = image_data['file_name']
        annotations[image_id] = {'file_name': file_name, 'objects': []}

    for annotation_data in data['annotations']:
        image_id = annotation_data['image_id']
        category_id = annotation_data['category_id']
        bbox = annotation_data['bbox']
        xmin, ymin, width, height = bbox
        xmax = xmin + width
        ymax = ymin + height
        annotations[image_id]['objects'].append({
            'category_id': category_id,
            'bbox': [xmin, ymin, xmax, ymax]
        })
    return annotations

# Example usage
coco_annotations = parse_coco_annotations('face cephalometry.v6i.coco/train/_annotations.coco.json')

# Set up your dataset
train_dataset = "face cephalometry.v6i.coco/train"  # Define your TensorFlow dataset for training
val_dataset = "face cephalometry.v6i.coco/valid"    # Define your TensorFlow dataset for validation

# Define input shape and number of classes
input_shape = (640, 640, 3)  # Adjust according to your image size
num_classes = 26  # Adjust according to the number of classes in your dataset

# Create and compile the model
model = create_ssd_model(input_shape, num_classes)
model.compile(optimizer=Adam(), loss=MeanSquaredError())

# Train the model
model.fit(train_dataset, validation_data=val_dataset, epochs=10)

# Save the trained model
model.save("multiple_class_object_detection_model.h5")


2024-04-02 14:36:02.670208: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-02 14:36:02.692052: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-02 14:36:02.812725: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-02 14:36:02.812796: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-02 14:36:02.833286: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

2024-04-02 14:36:06.779338: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-02 14:36:06.794914: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


IndexError: tuple index out of range

In [30]:
import os
import json
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Function to parse COCO annotations
def parse_coco_annotations(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)

    annotations = {}
    for image_data in data['images']:
        image_id = image_data['id']
        file_name = image_data['file_name']
        annotations[image_id] = {'image_id':image_id,'file_name': file_name, 'objects': []}

    for annotation_data in data['annotations']:
        image_id = annotation_data['image_id']
        category_id = annotation_data['category_id']
        bbox = annotation_data['bbox']
        xmin, ymin, width, height = bbox
        xmax = xmin + width
        ymax = ymin + height
        annotations[image_id]['objects'].append({
            'image_id': image_id, 
            'category_id': category_id,
            'bbox': [xmin, ymin, xmax, ymax]
        })
    return annotations

# Function to load and preprocess image
def load_and_preprocess_image(image_path):
    image = load_img(image_path,target_size=(224,224))  # Example: resize images to 224x224
    image = img_to_array(image) / 255.0  # Normalize pixel values to [0, 1]
    return image

# Path to the train folder and annotations file
train_folder = 'face cephalometry.v6i.coco/train'
annotations_file = "face cephalometry.v6i.coco/train/_annotations.coco.json"

# Load annotations
annotations = parse_coco_annotations(annotations_file)

# Create lists to store image paths and annotations
image_paths = []
image_annotations = []

i=0
# Iterate through annotations to collect image paths and annotations
for annotation in annotations.values():
    image_id = annotation['image_id']
    image_filename = annotation['file_name']
    image_path = os.path.join(train_folder, 'images', image_filename)
    image_paths.append(image_path)
    image_annotations.append(annotation)

# Load and preprocess images
images = [load_and_preprocess_image(image_path) for image_path in image_paths]

# Create TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices((images, image_annotations))


ValueError: Attempt to convert a value ({'image_id': 0, 'file_name': 'DSCF6710_JPG.rf.cbd0895783e1f3a4eb679fb856810ff5.jpg', 'objects': [{'image_id': 0, 'category_id': 1, 'bbox': [491, 227, 498.5, 232.5]}, {'image_id': 0, 'category_id': 15, 'bbox': [485, 247, 493, 252]}, {'image_id': 0, 'category_id': 7, 'bbox': [421, 247, 428, 251.5]}, {'image_id': 0, 'category_id': 23, 'bbox': [454, 236, 468, 239.5]}, {'image_id': 0, 'category_id': 5, 'bbox': [450, 241, 468.5, 245.5]}, {'image_id': 0, 'category_id': 12, 'bbox': [447, 258, 461.5, 261.5]}, {'image_id': 0, 'category_id': 14, 'bbox': [454, 248, 460.5, 252.5]}, {'image_id': 0, 'category_id': 18, 'bbox': [532, 309, 540, 314]}, {'image_id': 0, 'category_id': 25, 'bbox': [502, 311, 509.5, 315.5]}, {'image_id': 0, 'category_id': 2, 'bbox': [475, 317, 482, 322]}, {'image_id': 0, 'category_id': 1, 'bbox': [469, 329, 476, 333.5]}, {'image_id': 0, 'category_id': 22, 'bbox': [489, 329, 496.5, 333.5]}, {'image_id': 0, 'category_id': 13, 'bbox': [48

In [27]:
# Define your data loading pipeline
import json

def parse_coco_annotations(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)

    annotations = {}
    for image_data in data['images']:
        image_id = image_data['id']
        file_name = image_data['file_name']
        annotations[image_id] = {'file_name': file_name, 'objects': []}

    for annotation_data in data['annotations']:
        image_id = annotation_data['image_id']
        category_id = annotation_data['category_id']
        bbox = annotation_data['bbox']
        xmin, ymin, width, height = bbox
        xmax = xmin + width
        ymax = ymin + height
        annotations[image_id]['objects'].append({
            'category_id': category_id,
            'bbox': [xmin, ymin, xmax, ymax]
        })
    return annotations

# Example usage
coco_annotations = parse_coco_annotations('face cephalometry.v6i.coco/train/_annotations.coco.json')
print(coco_annotations)
for a in coco_annotations.values():
    print(a.keys()) 

{0: {'file_name': 'DSCF6710_JPG.rf.cbd0895783e1f3a4eb679fb856810ff5.jpg', 'objects': [{'category_id': 1, 'bbox': [491, 227, 498.5, 232.5]}, {'category_id': 15, 'bbox': [485, 247, 493, 252]}, {'category_id': 7, 'bbox': [421, 247, 428, 251.5]}, {'category_id': 23, 'bbox': [454, 236, 468, 239.5]}, {'category_id': 5, 'bbox': [450, 241, 468.5, 245.5]}, {'category_id': 12, 'bbox': [447, 258, 461.5, 261.5]}, {'category_id': 14, 'bbox': [454, 248, 460.5, 252.5]}, {'category_id': 18, 'bbox': [532, 309, 540, 314]}, {'category_id': 25, 'bbox': [502, 311, 509.5, 315.5]}, {'category_id': 2, 'bbox': [475, 317, 482, 322]}, {'category_id': 1, 'bbox': [469, 329, 476, 333.5]}, {'category_id': 22, 'bbox': [489, 329, 496.5, 333.5]}, {'category_id': 13, 'bbox': [489, 352, 496, 356.5]}, {'category_id': 4, 'bbox': [439, 368, 446, 373]}, {'category_id': 11, 'bbox': [472, 375, 479, 379.5]}, {'category_id': 16, 'bbox': [470, 402, 477.5, 407]}, {'category_id': 10, 'bbox': [443, 424, 450.5, 428.5]}, {'category_id

In [1]:
import os

from ultralytics import YOLO


# Load a model
model = YOLO("yolov8n.yaml")  # build a new model from scratch

# Use the model
results = model.train(data="/home/selvatharrun/Documents/GitHub/cephalometric-analysis/face cephalometry.v6i.yolov8/data.yaml", epochs=1)  # train the model


New https://pypi.org/project/ultralytics/8.1.42 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.29 🚀 Python-3.11.6 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 3902MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/home/selvatharrun/Documents/GitHub/cephalometric-analysis/face cephalometry.v6i.yolov8/data.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, 

2024-04-03 15:26:56.385916: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-03 15:26:56.385986: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-03 15:26:56.405464: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Overriding model.yaml nc=80 with nc=26

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

train: Scanning /home/selvatharrun/Documents/GitHub/cephalometric-analysis/face cephalometry.v6i.yolov8/train/labels.cache... 297 images, 0 backgrounds, 0 corrupt: 100%|██████████| 297/297 [00:00<?, ?it/s]
val: Scanning /home/selvatharrun/Documents/GitHub/cephalometric-analysis/face cephalometry.v6i.yolov8/valid/labels.cache... 82 images, 0 backgrounds, 0 corrupt: 100%|██████████| 82/82 [00:00<?, ?it/s]


Plotting labels to runs/detect/train4/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000333, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train4
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         82       1896          0          0          0          0



1 epochs completed in 0.003 hours.
Optimizer stripped from runs/detect/train4/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train4/weights/best.pt, 6.3MB

Validating runs/detect/train4/weights/best.pt...
Ultralytics YOLOv8.1.29 🚀 Python-3.11.6 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 3902MiB)
YOLOv8n summary (fused): 168 layers, 3010718 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  4.26it/s]

                   all         82       1896          0          0          0          0


Speed: 3.7ms preprocess, 3.1ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs/detect/train4
